# visualization of point clouds for unbounded scenes

In [5]:
import numpy as np
import matplotlib.pyplot as plt

import open3d as o3d

%matplotlib inline

In [6]:

root = '../logs/unbounded/outdoor1/outdoor1/'

cam_train = root + 'cam_train.npz'
cam_test = root + 'cam_test.npz'
path = root + 'fine.npz'
data = np.load(path)

In [7]:
# show the bounding box defined by cameras, here each cam:(5,3) represents the coordiantes of five points
if cam_train:
    cam_train = np.load(cam_train)
    xyz_min = cam_train['xyz_min']
    xyz_max = cam_train['xyz_max']
    print('xyz_min,xyz_max',xyz_min,xyz_max)
    cam_lst_train_ori = cam_train['cam_lst']
    cam_test = np.load(cam_test)
    cam_lst_test_ori = cam_test['cam_lst']
    cam_frustrm_lst = []
    # normalize camera to be [-1,1] for unbounded scenes
    scene_center = (xyz_min + xyz_max) * 0.5
    scene_radius = (scene_center - xyz_min).max()
    print('scene_center,scene_radius',scene_center,scene_radius)
    cam_lst_train_norm = (cam_lst_train_ori - scene_center) / scene_radius
    cam_lst_test_norm = (cam_lst_test_ori - scene_center) / scene_radius
    # select initial views
    print(cam_lst_train_norm.shape)
    for cam in cam_lst_train_norm:
        for i in range(1,len(cam)): # shorten the line length of the shown cam frustrm
            t = 0.2 # scale coefficient
            cam[i] = cam[i]*t + cam[0]*(1-t)
        cam_frustrm = o3d.geometry.LineSet()
        cam_frustrm.points = o3d.utility.Vector3dVector(cam)
        if len(cam) == 5:
            cam_frustrm.colors = o3d.utility.Vector3dVector([[0,0,1] for i in range(8)])
            cam_frustrm.lines = o3d.utility.Vector2iVector([[0,1],[0,2],[0,3],[0,4],[1,2],[2,4],[4,3],[3,1]])
        elif len(cam) == 8:
            cam_frustrm.colors = o3d.utility.Vector3dVector([[0.5,0.5,0.5] for i in range(12)])
            cam_frustrm.lines = o3d.utility.Vector2iVector([
                [0,1],[1,3],[3,2],[2,0],
                [4,5],[5,7],[7,6],[6,4],
                [0,4],[1,5],[3,7],[2,6],
            ])
        cam_frustrm_lst.append(cam_frustrm)
    for cam in cam_lst_test_norm:
        # for i in range(1,len(cam)): # shorten the line length of the shown cam frustrm
        #     t = 0.1 # scale coefficient
        #     cam[i] = cam[i]*t + cam[0]*(1-t)
        cam_frustrm = o3d.geometry.LineSet()
        cam_frustrm.points = o3d.utility.Vector3dVector(cam)
        if len(cam) == 5:
            cam_frustrm.colors = o3d.utility.Vector3dVector([[0.5,0,0] for i in range(8)])
            cam_frustrm.lines = o3d.utility.Vector2iVector([[0,1],[0,2],[0,3],[0,4],[1,2],[2,4],[4,3],[3,1]])
        elif len(cam) == 8:
            cam_frustrm.colors = o3d.utility.Vector3dVector([[0.5,0.5,0.5] for i in range(12)])
            cam_frustrm.lines = o3d.utility.Vector2iVector([
                [0,1],[1,3],[3,2],[2,0],
                [4,5],[5,7],[7,6],[6,4],
                [0,4],[1,5],[3,7],[2,6],
            ])
        # cam_frustrm_lst.append(cam_frustrm)
else:
    cam_frustrm_lst = []

# select initial views
# cam_frustrm_lst_init = [cam_frustrm_lst[i] for i in [0,1,2,3,4,5,6,7,61,62,63,64,65,66,67,112,113,114,115,116,117,118]]
# cam_frustrm_lst_init = [cam_frustrm_lst[i] for i in [14,15,16,17,18,19,20,21]]

aabb_01 = np.array([[0, 0, 0],
                    [0, 0, 1],
                    [0, 1, 1],
                    [0, 1, 0],
                    [1, 0, 0],
                    [1, 0, 1],
                    [1, 1, 1],
                    [1, 1, 0]])
out_bbox = o3d.geometry.LineSet()
out_bbox.points = o3d.utility.Vector3dVector(xyz_min + aabb_01 * (xyz_max - xyz_min))
out_bbox.colors = o3d.utility.Vector3dVector([[1,0,0] for i in range(12)])
out_bbox.lines = o3d.utility.Vector2iVector([[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],[0,4],[1,5],[2,6],[3,7]]) # connect points by this order

def change_background_to_black(vis):
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    return False

# o3d.visualization.draw_geometries_with_key_callbacks([
#     o3d.geometry.TriangleMesh.create_coordinate_frame(size=(xyz_max-xyz_min).min()*0.1, origin=xyz_min),
#     out_bbox, *cam_frustrm_lst,
# ], {ord("K"): change_background_to_black})

xyz_min,xyz_max [-0.71020436 -1.02989    -0.05864531] [0.6428573  0.32317173 1.2944164 ]
scene_center,scene_radius [-0.03367352 -0.3533591   0.6178856 ] 0.6765309
(25, 5, 3)


In [8]:
# show the bounding box defined by fine model 

xyz_fine_min = data['xyz_min_fine']
xyz_fine_max = data['xyz_max_fine']
print('xyz fine box:',xyz_fine_min, xyz_fine_max)

# display nonaffected space [-1,-1,-1] ~ [1,1,1]
out_bbox1 = o3d.geometry.LineSet()
out_bbox1.points = o3d.utility.Vector3dVector(aabb_01 * 2 - 1)
out_bbox1.colors = o3d.utility.Vector3dVector([[0,0,1] for i in range(12)])
out_bbox1.lines = o3d.utility.Vector2iVector([[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],[0,4],[1,5],[2,6],[3,7]]) # connect points by this order

out_bbox2 = o3d.geometry.LineSet()
out_bbox2.points = o3d.utility.Vector3dVector(xyz_fine_min + aabb_01 * (xyz_fine_max - xyz_fine_min))
out_bbox2.colors = o3d.utility.Vector3dVector([[0,1,0] for i in range(12)])
out_bbox2.lines = o3d.utility.Vector2iVector([[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],[0,4],[1,5],[2,6],[3,7]]) # connect points by this order

# 
N=201
world_size = np.array([N,N,N])
normalized_fbbox_lower = (np.array([-1,-1,-1]) - xyz_fine_min) / (xyz_fine_max - xyz_fine_min)
normalized_fbbox_upper = (np.array([1,1,1]) - xyz_fine_min) / (xyz_fine_max - xyz_fine_min)
fbbox_indices_lower = np.round(normalized_fbbox_lower * world_size).astype(np.long)
fbbox_indices_upper = np.round(normalized_fbbox_upper * world_size).astype(np.long)
print('fbbox_indices_lower upper',fbbox_indices_lower,fbbox_indices_upper)

# density points
target = data['alpha']
thres = 0.5

# target[:fbbox_indices_lower[0]] = 0
# target[fbbox_indices_upper[0]:] = 0
# target[:,:fbbox_indices_lower[1]] = 0
# target[:,fbbox_indices_upper[1]:] = 0
# target[:,:,:fbbox_indices_lower[2]] = 0
# target[:,:,fbbox_indices_upper[2]:] = 0

xyz = np.stack((target > thres).nonzero(), -1)
xyz_norm = xyz / target.shape * (xyz_fine_max - xyz_fine_min) + xyz_fine_min
print('xyz.shape:',xyz.shape)
color = data['rgb'][xyz[:,0], xyz[:,1], xyz[:,2]]

pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(xyz_norm)
pcd0.colors = o3d.utility.Vector3dVector(color)
voxel_grid_0 = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd0, voxel_size=max((xyz_fine_max - xyz_fine_min) / target.shape))
# voxel_grid_0 = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd0, voxel_size=0.005)

# uncertainty points
target, thres = data['uncertainty_VH'], 0
print('target',target.shape)

# target[:fbbox_indices_lower[0]] = 0
# target[fbbox_indices_upper[0]:] = 0
target[:,:fbbox_indices_lower[1]] = 0
target[:,fbbox_indices_upper[1]:] = 0
# target[:,:,:fbbox_indices_lower[2]] = 0
# target[:,:,fbbox_indices_upper[2]:] = 0

target[:,80:] = 0

# target = target[fbbox_indices_lower[0]:fbbox_indices_upper[0],fbbox_indices_lower[1]:fbbox_indices_upper[1],fbbox_indices_lower[2]:fbbox_indices_upper[2]]
# xyz_fine_max = np.array([1,1,1])
# xyz_fine_min = np.array([-1,-1,-1])

xyz_VH = np.stack((target > thres).nonzero(), -1)[::3]
xyz_VH_norm = xyz_VH / target.shape * (xyz_fine_max - xyz_fine_min) + xyz_fine_min
print('xyz_VH.shape:',xyz_VH.shape)
color_VH = [[0.5,0,0]]*xyz_VH.shape[0]

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(xyz_VH_norm)
pcd1.colors = o3d.utility.Vector3dVector(color_VH)
# voxel_grid_1 = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd1, voxel_size=max((xyz_fine_max - xyz_fine_min) / target.shape))
voxel_grid_1 = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd1, voxel_size=0.01)


def change_background_to_black(vis):
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    return False

# o3d.visualization.draw_geometries_with_key_callbacks([
#     o3d.geometry.TriangleMesh.create_coordinate_frame(size=(xyz_fine_max-xyz_fine_min).min()*0.1, origin=xyz_fine_min),
#     out_bbox1, out_bbox2, voxel_grid_1, *cam_frustrm_lst,
# ], {ord("K"): change_background_to_black})

o3d.visualization.draw_geometries_with_key_callbacks([
    o3d.geometry.TriangleMesh.create_coordinate_frame(size=(xyz_fine_max-xyz_fine_min).min()*0.1, origin=xyz_fine_min),
    voxel_grid_1, *cam_frustrm_lst,
], {ord("K"): change_background_to_black})

xyz fine box: [-1.2 -1.2 -1.2] [1.2 1.2 1.2]
fbbox_indices_lower upper [17 17 17] [184 184 184]


xyz.shape: (311896, 3)
